In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import f_oneway
from itertools import combinations
from scipy.stats import ttest_ind
import statsmodels.stats.multitest as smm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
df = pd.read_csv('comparison_cv.csv')

In [3]:
right = df[df['Limb'] == 'Right']
left = df[df['Limb'] == 'Left']
aff = df[df['Limb'] == 'Affected']
unaff = df[df['Limb'] == 'Unaffected']


In [4]:

def pair_tukeyhsd(df_limb, filename):
  model_names = ['RF_inter', 'GMAC_L', 'GMAC_G', 'Scheme1_L', 'Scheme1_G', 'Scheme1b_L', 'Scheme1b_G',
               'Scheme2_L', 'Scheme2_G', 'Scheme3a_L', 'Scheme3a_G', 'Scheme3b_L', 'Scheme3b_G']
  data = []
  for model in model_names:
      data.extend(zip(df_limb[model], [model] * len(df_limb[model])))
  df = pd.DataFrame(data, columns=['YI', 'Model'])
  tukey_results = pairwise_tukeyhsd(df['YI'], df['Model'], alpha=0.05)
  #print(tukey_results)
  tukey_df = pd.DataFrame({
              'group1': tukey_results.groupsunique[tukey_results._multicomp.pairindices[0]],
              'group2': tukey_results.groupsunique[tukey_results._multicomp.pairindices[1]], 
              'meandiff': tukey_results.meandiffs,
              'p-adj': tukey_results.pvalues,  
              'lower': tukey_results.confint[:, 0],
              'upper': tukey_results.confint[:, 1],
              'reject': tukey_results.reject
            })
  #tukey_df = pd.DataFrame(data=tukey_results._results_table.data[1:], columns=tukey_results._results_table.data[0])
  print(tukey_df)
  latex_table = tukey_df.to_latex(index=False, float_format="%.7f")
  with open(filename, "w") as f:
    f.write(latex_table)
  return tukey_df


In [5]:
print("Right Limb (One-way ANOVA)")
f_oneway(right['RF_inter'], right['GMAC_L'], right['GMAC_G'], right['Scheme1_L'], right['Scheme1_G'], right['Scheme1b_L'], right['Scheme1b_G'], right['Scheme2_L'], right['Scheme2_G'], right['Scheme3a_L'], right['Scheme3a_G'], right['Scheme3b_L'], right['Scheme3b_G'])


Right Limb (One-way ANOVA)


F_onewayResult(statistic=9.326965779805978, pvalue=1.9808997857478744e-12)

In [6]:
print("Right Limb (Post-hoc Tukey)")
right_tukey = pair_tukeyhsd(right, 'tukey_right.txt')

Right Limb (Post-hoc Tukey)
        group1      group2  meandiff     p-adj     lower     upper  reject
0       GMAC_G      GMAC_L    -0.056  0.999794 -0.285683  0.173683   False
1       GMAC_G    RF_inter     0.009  1.000000 -0.220683  0.238683   False
2       GMAC_G   Scheme1_G    -0.034  0.999999 -0.263683  0.195683   False
3       GMAC_G   Scheme1_L    -0.034  0.999999 -0.263683  0.195683   False
4       GMAC_G  Scheme1b_G     0.028  1.000000 -0.201683  0.257683   False
..         ...         ...       ...       ...       ...       ...     ...
73  Scheme3a_G  Scheme3b_G    -0.103  0.949460 -0.332683  0.126683   False
74  Scheme3a_G  Scheme3b_L     0.008  1.000000 -0.221683  0.237683   False
75  Scheme3a_L  Scheme3b_G    -0.206  0.126708 -0.435683  0.023683   False
76  Scheme3a_L  Scheme3b_L    -0.095  0.972481 -0.324683  0.134683   False
77  Scheme3b_G  Scheme3b_L     0.111  0.915181 -0.118683  0.340683   False

[78 rows x 7 columns]


In [7]:
print("Left limb (One-way ANOVA)")
f_oneway(left['RF_inter'], left['GMAC_L'], left['GMAC_G'], left['Scheme1_L'], left['Scheme1_G'], left['Scheme1b_L'], left['Scheme1b_G'],left['Scheme2_L'], left['Scheme2_G'], left['Scheme3a_L'], left['Scheme3a_G'], left['Scheme3b_L'], left['Scheme3b_G'])


Left limb (One-way ANOVA)


F_onewayResult(statistic=7.770534897376724, pvalue=1.801202593151408e-10)

In [8]:
print("Left Limb (Post-hoc Tukey)")
left_tukey = pair_tukeyhsd(left, 'tukey_left.txt')

Left Limb (Post-hoc Tukey)
        group1      group2  meandiff     p-adj     lower     upper  reject
0       GMAC_G      GMAC_L     0.065  0.997500 -0.142275  0.272275   False
1       GMAC_G    RF_inter     0.001  1.000000 -0.206275  0.208275   False
2       GMAC_G   Scheme1_G     0.004  1.000000 -0.203275  0.211275   False
3       GMAC_G   Scheme1_L     0.006  1.000000 -0.201275  0.213275   False
4       GMAC_G  Scheme1b_G     0.038  0.999990 -0.169275  0.245275   False
..         ...         ...       ...       ...       ...       ...     ...
73  Scheme3a_G  Scheme3b_G     0.005  1.000000 -0.202275  0.212275   False
74  Scheme3a_G  Scheme3b_L    -0.067  0.996682 -0.274275  0.140275   False
75  Scheme3a_L  Scheme3b_G    -0.128  0.671456 -0.335275  0.079275   False
76  Scheme3a_L  Scheme3b_L    -0.200  0.069746 -0.407275  0.007275   False
77  Scheme3b_G  Scheme3b_L    -0.072  0.993620 -0.279275  0.135275   False

[78 rows x 7 columns]


In [9]:
print("Affected limb (One-way ANOVA)")
f_oneway(aff['RF_inter'], aff['GMAC_L'], aff['GMAC_G'], aff['Scheme1_L'], aff['Scheme1_G'], aff['Scheme1b_L'], aff['Scheme1b_G'], aff['Scheme2_L'], aff['Scheme2_G'], aff['Scheme3a_L'], aff['Scheme3a_G'], aff['Scheme3b_L'], aff['Scheme3b_G'])


Affected limb (One-way ANOVA)


F_onewayResult(statistic=1.060137811520053, pvalue=0.4116242046067608)

In [10]:
print("Unaffected limb (One-way ANOVA)")
f_oneway(unaff['RF_inter'], unaff['GMAC_L'], unaff['GMAC_G'], unaff['Scheme1_L'], unaff['Scheme1_G'], unaff['Scheme1b_L'], unaff['Scheme1b_G'], unaff['Scheme2_L'], unaff['Scheme2_G'], unaff['Scheme3a_L'], unaff['Scheme3a_G'], unaff['Scheme3b_L'], unaff['Scheme3b_G'])

Unaffected limb (One-way ANOVA)


F_onewayResult(statistic=1.5797949345551692, pvalue=0.12688034636263879)

In [16]:
len(right_tukey), len(left_tukey)

(78, 78)

In [ ]:
combined_tukey = pd.merge(left_tukey, right_tukey, on=['group1', 'group2'], suffixes=('_left', '_right'))
combined_tukey.drop(columns=['reject_left', 'reject_right'], inplace=True)



In [ ]:
latex_table = combined_tukey.to_latex(index=False, float_format="%.7f")
with open('tukey_combined.txt', "w") as f:
    f.write(latex_table)

In [28]:
print(combined_tukey[(combined_tukey['group1']=='Scheme1b_L') & (combined_tukey['group2']=='Scheme3b_L')])

        group1      group2  meandiff_left  p-adj_left  lower_left  upper_left  \
62  Scheme1b_L  Scheme3b_L         -0.236    0.011605   -0.443275   -0.028725   

    meandiff_right  p-adj_right  lower_right  upper_right  
62          -0.235     0.039799    -0.464683    -0.005317  
